# Census Data

This notebook imports 4 csv files that contain census data for the 207 tracts in Suffolk County, MA. The data has been normalized (credit to Brown University) to reflect the tracts from the 2010 census. After a number of pre-processing steps, I calculate the percentage change between the two censuses with an end goal of using k-means clustering on the tracts.

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

## Data pre-processing (2000 census, full count)

In [826]:
# import file
df_2000 = pd.read_csv('CSV_files/LTDB_Std_2000_fullcount.csv',sep=',', engine='python')

# subset the file to Massachusetts
df_2000_MA = df_2000[(df_2000['state']=='MA')]

In [827]:
# collect tracts for Cambridge and Brookline
Cambridge_tracts = list((range(3522,3531)))+list(((range(3532,3551))))
Cambridge_tracts = Cambridge_tracts + [3521.01,3521.02,3531.02,3531.01]
Cambridge_tracts = [f'Census Tract {str(x)}' for x in Cambridge_tracts]

Brookline_tracts = list(range(4001,4013))
Brookline_tracts = [f'Census Tract {str(x)}' for x in Brookline_tracts]

Somerville_tracts = [3507,3508,3505,3504,3506,3503,3502,3501.04,3501.03,
                     3514.03,3512.03,3512.04,3513,3511,3510,3509,3515]
Somerville_tracts = [f'Census Tract {str(x)}' for x in Somerville_tracts]

In [828]:
# get Suffolk County
df_2000_MA_1 = df_2000_MA[df_2000_MA['county']=='Suffolk County']

# get Cambridge
df_2000_MA_2 =df_2000_MA[df_2000_MA['tract'].apply(lambda x: any(tract for tract in Cambridge_tracts if tract in x))]

# get Brookline
df_2000_MA_3=df_2000_MA[df_2000_MA['tract'].apply(lambda x: any(tract for tract in Brookline_tracts if tract in x))]

# get Somerville
df_2000_MA_4=df_2000_MA[df_2000_MA['tract'].apply(lambda x: any(tract for tract in Somerville_tracts if tract in x))]

In [829]:
print(df_2000_MA_1.shape)
print(df_2000_MA_2.shape)
print(df_2000_MA_3.shape)
print(df_2000_MA_4.shape)

(203, 51)
(32, 51)
(12, 51)
(17, 51)


In [830]:
# merge the 3
df_MA_full_2000= pd.concat([df_2000_MA_1,df_2000_MA_2,df_2000_MA_3,df_2000_MA_4],ignore_index=True)

# drop irrelevant columns
df_MA_full_2000.drop(columns=['placefp10','cbsa10','metdiv10','ccflag10'],axis=1,inplace=True)

# reset the index
df_MA_full_2000.reset_index(inplace=True, drop=True)

In [831]:
# rename tract id column to match 2010
df_MA_full_2000.rename(columns={'TRTID10':'tractid'},inplace=True)

## Feature engineering

In [832]:
# calculate percentages of population for each category 
df_MA_full_2000['percent_white00']=100*df_MA_full_2000['NHWHT00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_black00']=100*df_MA_full_2000['NHBLK00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_asian00']=100*df_MA_full_2000['ASIAN00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_hispanic00']=100*df_MA_full_2000['HISP00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_indian00']=100*df_MA_full_2000['INDIA00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_chinese00']=100*df_MA_full_2000['CHINA00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_filip00']=100*df_MA_full_2000['FILIP00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_japan00']=100*df_MA_full_2000['JAPAN00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_korean00']=100*df_MA_full_2000['KOREA00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_viet00']=100*df_MA_full_2000['VIET00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_mex00']=100*df_MA_full_2000['MEX00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_pr00']=100*df_MA_full_2000['PR00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_cuban00']=100*df_MA_full_2000['CUBAN00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_vacant_housing00']=100*df_MA_full_2000['VAC00']/df_MA_full_2000['HU00']
df_MA_full_2000['percent_occupied_housing00']=100*df_MA_full_2000['OHU00']/df_MA_full_2000['HU00']
df_MA_full_2000['percent_under18_00']=100*df_MA_full_2000['A18UND00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_60andup_00']=100*df_MA_full_2000['A60UP00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_75andup_00']=100*df_MA_full_2000['A75UP00']/df_MA_full_2000['POP00']
df_MA_full_2000['percent_owneroccupied_00']=100*df_MA_full_2000['OWN00']/df_MA_full_2000['HU00']
df_MA_full_2000['percent_renteroccupied_00']=100*df_MA_full_2000['RENT00']/df_MA_full_2000['HU00']

In [833]:
# let's recreate the race buckets into "percent white" and "percent non-white"
df_MA_full_2000['percent_non-white00'] = 100*(df_MA_full_2000.iloc[:,6:20].sum(1)/df_MA_full_2000['POP00'])

In [834]:
# now lets drop all other columns
df_MA_full_2000.drop(df_MA_full_2000.columns[5:47],axis=1,inplace=True)

## Data pre-processing (2010 census, full count)

In [835]:
df_2010 = pd.read_csv('CSV_files/LTDB_Std_2010_fullcount.csv',sep=',', engine='python')

In [836]:
df_2010_MA = df_2010[(df_2010['state']=='MA')]

In [837]:
# same steps for 2010 

# get Suffolk County
df_2010_MA_1 = df_2010_MA[df_2010_MA['county']=='Suffolk County']

# get Cambridge
df_2010_MA_2 =df_2010_MA[df_2010_MA['tract'].apply(lambda x: any(tract for tract in Cambridge_tracts if tract in x))]

# get Brookline
df_2010_MA_3=df_2010_MA[df_2010_MA['tract'].apply(lambda x: any(tract for tract in Brookline_tracts if tract in x))]

# get Somerville
df_2010_MA_4=df_2010_MA[df_2010_MA['tract'].apply(lambda x: any(tract for tract in Somerville_tracts if tract in x))]

In [838]:
print(df_2010_MA_1.shape)
print(df_2010_MA_2.shape)
print(df_2010_MA_3.shape)
print(df_2010_MA_4.shape)

(204, 47)
(32, 47)
(12, 47)
(17, 47)


In [839]:
# merge the 3
df_MA_full_2010= pd.concat([df_2010_MA_1,df_2010_MA_2,df_2010_MA_3,df_2010_MA_4],ignore_index=True)

# reset the index
df_MA_full_2010.reset_index(inplace=True, drop=True)

In [840]:
# calculate percentages of population for each category 
df_MA_full_2010['percent_white10']=100*df_MA_full_2010['nhwht10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_black10']=100*df_MA_full_2010['nhblk10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_asian10']=100*df_MA_full_2010['asian10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_hispanic10']=100*df_MA_full_2010['hisp10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_indian10']=100*df_MA_full_2010['india10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_chinese10']=100*df_MA_full_2010['china10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_filip10']=100*df_MA_full_2010['filip10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_japan10']=100*df_MA_full_2010['japan10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_korean10']=100*df_MA_full_2010['korea10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_viet10']=100*df_MA_full_2010['viet10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_mex10']=100*df_MA_full_2010['mex10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_pr10']=100*df_MA_full_2010['pr10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_cuban10']=100*df_MA_full_2010['cuban10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_vacant_housing10']=100*df_MA_full_2010['vac10']/df_MA_full_2010['hu10']
df_MA_full_2010['percent_occupied_housing10']=100*df_MA_full_2010['ohu10']/df_MA_full_2010['hu10']
df_MA_full_2010['percent_under18_10']=100*df_MA_full_2010['a18und10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_60andup_10']=100*df_MA_full_2010['a60up10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_75andup_10']=100*df_MA_full_2010['a75up10']/df_MA_full_2010['pop10']
df_MA_full_2010['percent_owneroccupied_10']=100*df_MA_full_2010['own10']/df_MA_full_2010['hu10']
df_MA_full_2010['percent_renteroccupied_10']=100*df_MA_full_2010['rent10']/df_MA_full_2010['hu10']

In [841]:
# let's recreate the race buckets into "white" and "non-white"
df_MA_full_2010['non-white10'] = 100*(df_MA_full_2010.iloc[:,6:20].sum(1)/df_MA_full_2010['pop10'])

In [842]:
# now lets drop all other columns
df_MA_full_2010.drop(df_MA_full_2010.columns[5:47],axis=1,inplace=True)

## Calculate change b/w censuses (full count)

In [843]:
## New df for change in values between 2000 and 2010
df_joined = df_MA_full_2000.merge(df_MA_full_2010, on='tractid')

In [844]:
df_joined.drop(columns=['state_y','county_y','tract_y'],axis=1,inplace=True)

In [845]:
# 2000 starts at column 4
i = 4
# 2010 starts at column
j = 26
# stop once you get to final column
while j < 49:
    try:
        df_joined[f'{df_joined.columns[i]}_PC'] = df_joined.iloc[:,1:].apply(lambda x: 100*
                                    ((x[df_joined.columns[j]]-x[df_joined.columns[i]])
                                   /x[df_joined.columns[i]]),axis=1)
    except ZeroDivisionError:
        0
        
    i +=1
    j +=1

In [846]:
df_joined.to_csv('CSV_files/Full_count_census_changes.csv')

## Data pre-processing (2000 Census, Sample count)

In [862]:
# import the file
df_2000_2 = pd.read_csv('CSV_files/LTDB_Std_2000_Sample.csv',sep=',', engine='python')

# subset to MA
df_2000_MA_2 = df_2000_2[(df_2000_2['state']=='MA')]

In [863]:
# get Suffolk County
df_MA_sample_2000_1 = df_2000_MA_2[df_2000_MA_2['county']=='Suffolk County']

# get Cambridge
df_MA_sample_2000_2=df_2000_MA_2[df_2000_MA_2['tract'].apply(lambda x: any(tract for tract in Cambridge_tracts if tract in x))]

# get Brookline
df_MA_sample_2000_3=df_2000_MA_2[df_2000_MA_2['tract'].apply(lambda x: any(tract for tract in Brookline_tracts if tract in x))]

# get Somerville
df_MA_sample_2000_4=df_2000_MA_2[df_2000_MA_2['tract'].apply(lambda x: any(tract for tract in Somerville_tracts if tract in x))]

In [864]:
print(df_MA_sample_2000_1.shape)
print(df_MA_sample_2000_2.shape)
print(df_MA_sample_2000_3.shape)
print(df_MA_sample_2000_4.shape)

(203, 77)
(32, 77)
(12, 77)
(17, 77)


In [865]:
# merge the 3
df_MA_sample_2000= pd.concat([df_MA_sample_2000_1,df_MA_sample_2000_2,df_MA_sample_2000_3,df_MA_sample_2000_4],ignore_index=True)

# reset the index
df_MA_sample_2000.reset_index(inplace=True, drop=True)

# drop irrelevant columns
df_MA_sample_2000.drop(columns=['placefp10','cbsa10','metdiv10','ccflag10'],axis=1,inplace=True)

In [866]:
df_MA_sample_2000.shape

(264, 73)

In [867]:
df_MA_sample_2000.drop(columns=['state','county','tract'],axis=1,inplace=True)

# Feature engineering

In [853]:
df_MA_sample_2000.head()

,TRTID10,POP00SF3,RUANC00,ITANC00,GEANC00,IRANC00,SCANC00,RUFB00,ITFB00,GEFB00,IRFB00,SCFB00,FB00,NAT00,N10IMM00,AG5UP00,OLANG00,LEP00,AG25UP00,HS00,COL00,AG15UP00,Mar-00,WDS00,CLF00,UNEMP00,DFLABF00,FLABF00,EMPCLF00,PROF00,MANUF00,SEMP00,AG18CV00,VET00,CNI16U00,DIS00,DPOV00,NPOV00,N65POV00,DFMPOV00,NFMPOV00,DWPOV00,NWPOV00,DBPOV00,NBPOV00,DNAPOV00,NNAPOV00,DHPOV00,NHPOV00,DAPOV00,NAPOV00,INCPC00,HU00SP,H30OLD00,OHU00SP,H10YRS00,DMULTI00,MULTI00,HINC00,HINCW00,HINCB00,HINCH00,HINCA00,MHMVAL00,MRENT00,HH00,HHW00,HHB00,HHH00,HHA00
0,25025000100,3968.000000,99.000000,528.000000,94.000000,636.000000,34.000000,24.000000,42.000000,7.0,55.000000,0.0,1033.000000,447.000000,417.00000,3752.000000,1290.000000,306.000000,2752.00000,1206.000000,1128.000000,3478.00000,1135.000000,416.000000,2446.000000,102.000000,1683.000000,1112.000000,2344.00000,982.000000,131.000000,266.00000,3414.000000,257.000000,3005.00000,588.000000,3929.000000,529.000000,47.000000,700.000000,31.000000,2819.000000,433.000000,143.000000,13.0,31.000000,0.0,327.00000,56.0,423.000000,7.0,21999.00000,1614.000000,1314.000000,1563.000000,1064.000000,1614.000000,1379.000000,44630,42404,45000,57833,63500,222100,898,1562.000000,1227.000000,56.000000,108.000000,107.000000
1,25025000201,3887.000000,82.000000,272.000000,83.000000,967.000000,40.000000,12.000000,21.000000,14.0,219.000000,7.0,1001.000000,457.000000,446.00000,3774.000000,1007.000000,209.000000,2771.00000,808.000000,1312.000000,3526.00000,1055.000000,290.000000,2805.000000,54.000000,1747.000000,1342.000000,2751.00000,1351.000000,223.000000,173.00000,3455.000000,181.000000,3131.00000,399.000000,3875.000000,337.000000,19.000000,680.000000,19.000000,2944.000000,210.000000,140.000000,43.0,0.000000,0.0,286.00000,40.0,304.000000,9.0,27675.00000,1574.000000,1458.000000,1543.000000,1128.000000,1574.000000,1272.000000,62305,65950,50568,45455,71250,265600,991,1550.000000,1313.000000,27.000000,80.000000,86.000000
2,25025000202,3925.000000,43.000000,246.000000,140.000000,1247.000000,31.000000,0.000000,10.000000,0.0,376.000000,0.0,1305.000000,601.000000,565.00000,3709.000000,1365.000000,323.000000,2708.00000,1173.000000,1051.000000,3299.00000,1319.000000,377.000000,2283.000000,62.000000,1700.000000,1113.000000,2221.00000,1066.000000,172.000000,127.00000,3184.000000,215.000000,2769.00000,492.000000,3920.000000,478.000000,34.000000,782.000000,57.000000,2505.000000,164.000000,202.000000,37.0,16.000000,2.0,443.00000,101.0,576.000000,102.0,23602.00000,1516.000000,1463.000000,1494.000000,959.000000,1516.000000,1212.000000,56803,58370,90027,28417,53548.3871,276700,820,1494.000000,1140.000000,55.000000,105.000000,146.000000
3,25025000301,2833.000000,91.000000,303.000000,111.000000,746.000000,16.000000,67.000000,69.000000,0.0,175.000000,11.0,691.000000,346.000000,324.00000,2665.000000,579.000000,107.000000,2165.00000,791.000000,966.000000,2496.00000,830.000000,371.000000,1669.000000,27.000000,1380.000000,910.000000,1642.00000,876.000000,65.000000,115.00000,2457.000000,181.000000,2015.00000,352.000000,2671.000000,240.000000,18.000000,507.000000,29.000000,2200.000000,157.000000,69.000000,0.0,29.000000,8.0,94.00000,59.0,196.000000,16.0,24920.00000,1132.000000,977.000000,1132.000000,725.000000,1132.000000,895.000000,56321,59213,31250,6250,49166.66667,214400,971,1159.000000,983.000000,22.000000,30.000000,78.000000
4,25025000302,2938.520266,208.096787,348.441132,95.821312,718.175903,39.683575,83.238716,26.133085,0.0,157.766403,0.0,747.212647,488.785492,214.87204,2872.703613,611.707764,168.413223,2209.69751,572.024231,1268.906494,2703.32251,909.818543,307.789673,2045.155884,71.624008,1374.406738,1026.933472,1973.53186,1166.309937,189.706848,113.24337,2687.836182,189.706848,2358.75293,190.674728,2938.520266,163.573761,8.711028,606.868286,9.678921,2463.285156,163.573761,15.486273,0.0,10.646812,0.0,70.65612,0.0,279.720795,0.0,35207.00128,1304.718506,1041.451904,1272.778076,794.639343,1304.718506,1061.777588,64749.99973,66613.00168,51250,108970.0021,

In [868]:
# drop ethnic groups
df_MA_sample_2000.drop(df_MA_sample_2000.columns[2:12],axis=1,inplace=True)

In [869]:
# drop columns that are not variables in 2010
df_MA_sample_2000.drop(columns=['DIS00','CNI16U00'],axis=1,inplace=True)

# feature engineering

In [870]:
# percentage
df_MA_sample_2000['percent_foreign_born']=100*df_MA_sample_2000['FB00']/df_MA_sample_2000['POP00SF3']
df_MA_sample_2000['percent_naturalized']=100*df_MA_sample_2000['NAT00']/df_MA_sample_2000['POP00SF3']
df_MA_sample_2000['percent_recent_immigrants(10)']=100*df_MA_sample_2000['N10IMM00']/df_MA_sample_2000['POP00SF3']
df_MA_sample_2000['percent_other_languages']=100*df_MA_sample_2000['OLANG00']/df_MA_sample_2000['AG5UP00']
df_MA_sample_2000['percent_hs_degree_orless']=100*df_MA_sample_2000['HS00']/df_MA_sample_2000['AG25UP00']
df_MA_sample_2000['percent_4yrcollege_degree_ormore']=100*df_MA_sample_2000['COL00']/df_MA_sample_2000['AG25UP00']
df_MA_sample_2000['percent_married']=100*df_MA_sample_2000['Mar-00']/df_MA_sample_2000['AG15UP00']
df_MA_sample_2000['percent_unemployed']=100*df_MA_sample_2000['UNEMP00']/df_MA_sample_2000['CLF00']
df_MA_sample_2000['percent_employed']=100*df_MA_sample_2000['EMPCLF00']/df_MA_sample_2000['CLF00']
df_MA_sample_2000['percent_professional']=100*df_MA_sample_2000['PROF00']/df_MA_sample_2000['EMPCLF00']
df_MA_sample_2000['percent_manufacturing']=100*df_MA_sample_2000['MANUF00']/df_MA_sample_2000['EMPCLF00']
df_MA_sample_2000['percent_self_employed']=100*df_MA_sample_2000['SEMP00']/df_MA_sample_2000['EMPCLF00']
df_MA_sample_2000['percent_poverty']=100*df_MA_sample_2000['NPOV00']/df_MA_sample_2000['DPOV00']
df_MA_sample_2000['percent_houses_30yrsago']=100*df_MA_sample_2000['H30OLD00']/df_MA_sample_2000['DMULTI00']
df_MA_sample_2000['percent_multiunit_houses']=100*df_MA_sample_2000['MULTI00']/df_MA_sample_2000['DMULTI00']

In [871]:
# drop the remaining columns
# only keep: per capita income , median household rent, median household income
df_MA_sample_2000.drop(df_MA_sample_2000.columns[1:39],axis=1,inplace=True)

In [872]:
df_MA_sample_2000.drop(df_MA_sample_2000.columns[2:13],axis=1,inplace=True)

In [873]:
df_MA_sample_2000.drop(df_MA_sample_2000.columns[4:9],axis=1,inplace=True)

In [875]:
### convert dtypes to floats so we can do math
df_MA_sample_2000['MHMVAL00'] = pd.to_numeric(df_MA_sample_2000['MHMVAL00'],errors='coerce')
df_MA_sample_2000['MRENT00'] = pd.to_numeric(df_MA_sample_2000['MRENT00'],errors='coerce')

In [876]:
# match id column to 2010
df_MA_sample_2000.rename(columns={'TRTID10':'tractid'},inplace=True)

## Data pre-processing (2010 Census, Sample count)

In [877]:
# import the file
df_2010_2 = pd.read_csv('CSV_files/LTDB_Std_2010_Sample.csv',sep=',', engine='python')

# subset to MA
df_2010_2_MA= df_2010_2[(df_2010_2['statea']==25)]

In [878]:
# formatting is slightly different because this data is from the American Community Survey
# so we need to amend our lists
Cambridge_tracts_ACS = list((range(3522,3531)))+list(((range(3532,3551))))
Cambridge_tracts_ACS = [str(x)+'00' for x in Cambridge_tracts_ACS]
Cambridge_tracts_ACS = Cambridge_tracts_ACS + ['352101','352102','353102','353101']

Brookline_tracts_ACS = list(range(4001,4013))
Brookline_tracts_ACS = [str(x)+'00' for x in Brookline_tracts_ACS]

Somerville_tracts_ACS = [3507,3508,3505,3504,3506,3503,3502,3513,3511,3510,3509,3515,3501.04,3501.03,3514.03,3512.03,3512.04]
Somerville_tracts_ACS = [str(x).replace(".","") if str(x).find('.') != -1 else str(x)+'00' for x in Somerville_tracts_ACS]

In [879]:
df_2010_2_MA['tracta'] = df_2010_2_MA['tracta'].astype('str')

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [880]:
# get Suffolk County
df_MA_sample_2010_1 = df_2010_2_MA[df_2010_2_MA['countya']==25]

# get Cambridge
df_MA_sample_2010_2=df_2010_2_MA[df_2010_2_MA['tracta'].apply(lambda x: any(tract for tract in Cambridge_tracts_ACS if tract in x))]

# get Brookline
df_MA_sample_2010_3=df_2010_2_MA[df_2010_2_MA['tracta'].apply(lambda x: any(tract for tract in Brookline_tracts_ACS if tract in x))]

# get Somervile
df_MA_sample_2010_4=df_2010_2_MA[df_2010_2_MA['tracta'].apply(lambda x: any(tract for tract in Somerville_tracts_ACS if tract in x))]

In [881]:
print(df_MA_sample_2010_1.shape)
print(df_MA_sample_2010_2.shape)
print(df_MA_sample_2010_3.shape)
print(df_MA_sample_2010_4.shape)

(204, 176)
(32, 176)
(12, 176)
(17, 176)


In [882]:
# merge the 3
df_MA_sample_2010= pd.concat([df_MA_sample_2010_1,df_MA_sample_2010_2,df_MA_sample_2010_3,df_MA_sample_2010_4],ignore_index=True)

# reset the index
df_MA_sample_2010.reset_index(inplace=True, drop=True)

In [883]:
df_MA_sample_2010.shape

(265, 176)

## Feature engineering

In [884]:
# feature engineer the one column that is missing
df_MA_sample_2010['percent_employed12']=100*(df_MA_sample_2010['empclf12']/df_MA_sample_2010['clf12'])

In [885]:
# force these to be in the same order as the 2000 sample file
df_MA_sample_2010 = df_MA_sample_2010[['tractid','incpc12','mhmval12','mrent12','pfb12','pnat12','p10imm12','polang12','phs12','pcol12','pmar12',
                 'punemp12','percent_employed12','pprof12','pmanuf12','psemp12','ppov12','p30old12','pmulti12']]

## Calculate change b/w censuses (sample count)

In [886]:
# merge so we can calculate % change
df_joined_sample = df_MA_sample_2000.merge(df_MA_sample_2010, on='tractid')

In [887]:
df_joined_sample.shape

(264, 37)

In [888]:
# this is where 2000 data starts 
i = 1
# this is where 2010 data starts
j = 19
# stop once you get to final column
while j < 38:

    df_joined_sample[f'{df_joined_sample.columns[i]}_PC'] = df_joined_sample.iloc[:,1:].apply(lambda x: 100*
                                    ((x[df_joined_sample.columns[j]]-x[df_joined_sample.columns[i]])
                                     /x[df_joined_sample.columns[i]]),axis=1)
        
    i +=1
    j +=1

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [889]:
df_joined_sample.head()

,tractid,INCPC00,MHMVAL00,MRENT00,percent_foreign_born,percent_naturalized,percent_recent_immigrants(10),percent_other_languages,percent_hs_degree_orless,percent_4yrcollege_degree_ormore,percent_married,percent_unemployed,percent_employed,percent_professional,percent_manufacturing,percent_self_employed,percent_poverty,percent_houses_30yrsago,percent_multiunit_houses,incpc12,mhmval12,mrent12,pfb12,pnat12,p10imm12,polang12,phs12,pcol12,pmar12,punemp12,percent_employed12,pprof12,pmanuf12,psemp12,ppov12,p30old12,pmulti12,INCPC00_PC,MHMVAL00_PC,MRENT00_PC,percent_foreign_born_PC,percent_naturalized_PC,percent_recent_immigrants(10)_PC,percent_other_languages_PC,percent_hs_degree_orless_PC,percent_4yrcollege_degree_ormore_PC,percent_married_PC,percent_unemployed_PC,percent_employed_PC,percent_professional_PC,percent_manufacturing_PC,percent_self_employed_PC,percent_poverty_PC,percent_houses_30yrsago_PC,percent_multiunit_houses_PC,incpc12_PC
0,25025000100,21999.00000,222100.0000,898.000000,26.033266,11.265121,10.509073,34.381663,43.822674,40.988372,32.633698,4.170074,95.829926,41.894198,5.588737,11.348123,13.463986,81.412639,85.439901,27564,348200,1348,29.030001,15.30,9.48,38.450001,54.939999,30.780001,27.690001,6.06,94.380531,22.500000,7.83,7.13,14.660000,94.529999,78.639999,25.296604,56.776227,50.111359,11.511174,35.817451,-9.792235,11.832870,25.368886,-24.905530,-15.149056,45.321175,-1.512466,-46.293279,40.103205,-37.170225,8.883061,16.112190,-7.958695,-99.908226
1,25025000201,27675.00000,265600.0000,991.000000,25.752508,11.757139,11.474145,26.682565,29.159148,47.347528,29.920590,1.925134,98.074866,49.109415,8.106143,6.288622,8.696774,92.630241,80.813215,38128,532000,1405,25.129999,14.55,7.07,29.030001,32.290001,53.320000,27.059999,1.89,98.404032,45.750000,5.98,8.02,10.130000,95.980003,83.279999,37.770551,100.301205,41.775984,-2.417276,23.754597,-38.383205,8.797639,10.737119,12.614115,-9.560608,-1.825001,0.335627,-6.840674,-26.228789,27.531915,16.479972,3.616272,3.052451,-99.900937
2,25025000202,23602.00000,276700.0000,820.000000,33.248408,15.312102,14.394904,36.802373,43.316100,38.810931,39.981813,2.715725,97.284275,47.996398,7.744259,5.718145,12.193878,96.503958,79.947230,29292,432800,1299,36.240002,17.66,12.23,41.509998,32.820000,43.119999,33.090000,9.60,91.285081,45.950001,1.62,8.85,18.200001,96.260002,78.570000,24.108126,56.414890,58.414634,8.997706,15.333610,-15.039384,12.791636,-24.231407,11.102718,-17.237369,253.496788,-6.166663,-4.263648,-79.081279,54.770479,49.255236,-0.252793,-1.722674,-99.917697
3,25025000301,24920.00000,214400.0000,971.000000,24.391105,12.213202,11.436640,21.726079,36.535797,44.618938,33.253205,1.617735,98.382265,53.349574,3.958587,7.003654,8.985399,86.307420,79.063604,41955,409700,1469,15.200000,6.10,6.10,19.830000,26.520000,61.369999,34.970001,4.36,97.350649,63.820000,5.92,7.74,8.990000,93.970001,76.589996,68.358748,91.091418,51.287333,-37.682200,-50.054047,-46.662655,-8.727202,-27.413652,37.542492,5.162799,169.512601,-1.048579,19.626072,49.548310,10.513736,0.051206,8.878241,-3.128630,-99.837067
4,25025000302,35207.00128,259499.9961,1240.999993,25.428194,16.633729,7.312253,21.293800,25.886993,57.424443,33.655568,3.502130,96.497870,59.097599,9.612556,5.738107,5.566535,79.821962,81.379821,35219,322200,1436,34.490002,13.92,17.90,42.180000,31.459999,56.759998,33.619999,3.02,97.389969,56.020000,5.47,2.68,16.740000,94.820000,75.019997,0.034080,24.161852,15.713135,35.636849,-16.314614,144.794584,98.085829,21.528211,-1.157077,-0.105685,-13.766754,0.924476,-5.207653,-43.095260,-53.294700,200.725668,18.789363,-7.814990,-99.999903


In [890]:
df_joined_sample.to_csv('CSV_files/Sample_count_census_changes.csv')